## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [1]:
import random
lines = list(open('data/news_train.txt', 'r', encoding='utf-8'))
# random.shuffle(lines)

In [2]:
from collections import Counter

Counter([line.split('\t')[0] for line in lines[:15000]])

Counter({'sport': 2215,
         'culture': 2053,
         'science': 2156,
         'media': 2111,
         'economics': 2080,
         'life': 2033,
         'forces': 1225,
         'travel': 289,
         'style': 284,
         'business': 554})

In [3]:
import gensim
import string
import numpy as np

from tqdm.notebook import tqdm


from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import RussianStemmer 
rs = RussianStemmer()


In [4]:
stop_words = stopwords.words('russian')
stop_words.extend(['что', 'это', 'так', 'вот', 'быть','уже', 'как', 'в', '—', '–', 'к', 'на', '...','»', '«'])
stop_words.extend(list(string.punctuation))

def tokenize_sent2words_ru(sentence):
    sentence=sentence.lower()
    tokens = word_tokenize(sentence, 'russian')
    tokens = [rs.stem(i) for i in tokens if (i not in stop_words)]
    return tokens


def tokenize_new2item_ru(news):
    res = {}
    res["label"] = news[0]
    res["headline"] = tokenize_sent2words_ru(news[1])
    res["text"] = [tokenize_sent2words_ru(sent) for sent in sent_tokenize(news[2], 'russian')]
    return res

In [5]:
data_train = [tokenize_new2item_ru(line.split('\t')) for line in tqdm(lines)]

In [6]:
lines = list(open('data/news_test.txt', 'r', encoding='utf-8'))
data_test = [tokenize_new2item_ru(line.split('\t')) for line in tqdm(lines)]

In [7]:
sentences = [news["headline"] for news in data_train]
sentences.extend([sentance for news in data_train for sentance in news["text"]])

w2v = gensim.models.Word2Vec(sentences, workers=8)

In [8]:
w2v.save('./w2v.model')

**Тест Word2Vec**

In [9]:
pos = "комикс автор"

pos = [rs.stem(i) for i in pos.split()]
w2v.wv.most_similar(positive=pos)

[('мотив', 0.8740442395210266),
 ('роулинг', 0.8670545816421509),
 ('жанр', 0.866157054901123),
 ('исполнител', 0.865673840045929),
 ('создател', 0.8604228496551514),
 ('соавтор', 0.8585402965545654),
 ('клип', 0.8485084772109985),
 ('биограф', 0.8415507674217224),
 ('экранизац', 0.8393741250038147),
 ('доктор', 0.8378170132637024)]

In [10]:
w2v.wv["комикс"]

array([-0.6111654 , -1.03279   ,  0.12499405,  0.27501333,  1.1217716 ,
        0.6703801 , -0.29492298, -1.104037  , -0.4025427 ,  1.066527  ,
        0.13182026, -1.0439923 , -0.48487514,  0.79405916, -0.73271024,
        0.14954701, -0.02759554, -0.8746702 ,  0.02839555,  0.4093346 ,
       -0.06433468, -0.7363181 , -0.3728251 , -0.8569177 , -0.48993734,
       -0.10855093, -0.32718545, -0.61845285,  0.3402506 , -0.10560485,
       -0.95630985, -0.21765348,  0.4720715 , -0.03948889,  1.1397319 ,
       -0.08143106,  0.14216675, -0.03587299,  0.80415267,  0.18334499,
        1.121621  , -0.04954719, -0.98348176, -0.44081163,  0.20032005,
       -0.07225016,  0.7067896 ,  0.04029288,  0.11653626, -0.02766501,
        0.6469598 , -0.65641   ,  0.45214605,  0.13661584, -0.26133263,
       -0.04225916, -0.34904107,  0.6524689 , -0.7926059 ,  0.32990012,
        0.9036377 , -0.180573  ,  0.00900524,  0.25587705,  0.08089618,
       -0.08837476, -0.52629673,  0.06360698,  0.56004035, -0.19

# Классификация

In [12]:
max_len = 0

for item in data_train:
    l = len(item["headline"])+len(item["text"])
    if l>max_len: 
        max_len = l

max_len

67

In [13]:
label2idx = {}
max_item_len = max_len

def prepere_tfidf(data):
    X = []
    y = []

    for item in tqdm(data):
        label = item["label"]
        headline = item["headline"]
        text = item["text"]

        label_idx = label2idx.get(label, len(label2idx))
        label2idx[label] = label_idx

        word_idx = 0
        sent_idx = 0
        pos_in_sent = 0

        x = []

        while word_idx < max_item_len:
            if word_idx < len(headline):
                x.append(headline[word_idx])
                word_idx += 1
            else:
                if pos_in_sent < len(text[sent_idx]):
                    x.append(text[sent_idx][pos_in_sent])
                    word_idx += 1
                    pos_in_sent += 1
                elif sent_idx < len(text) - 1:
                    sent_idx += 1
                    pos_in_sent = 0
                else:
                    x.append("PLACEHOLDER")
                    word_idx += 1
        
        x = " ".join(x)
        X.append(x)
        y.append(label_idx)
    
    
    return X, y

In [14]:
X_train_idf, y_train = prepere_tfidf(data_train)

In [15]:
X_test_idf, y_test = prepere_tfidf(data_test)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_train_idf = tfidf.fit_transform(X_train_idf)
X_test_idf = tfidf.transform(X_test_idf)

In [17]:
X_train_idf.shape

(15000, 65785)

**Наивный байесовский классификатор**

In [18]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train_idf, y_train)
preds = clf.predict(X_test_idf)
print("accuracy =", (y_test == preds).mean())

accuracy = 0.7983333333333333


**Логистическая регрессия**

In [19]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(n_jobs=-1)
clf.fit(X_train_idf, y_train)
preds = clf.predict(X_test_idf)
print("accuracy =",(y_test == preds).mean())

accuracy = 0.8593333333333333
